# Setup

This notebook introduces the basic usage of the `RigidBodySim` package. Let's get started.

In [1]:
using RigidBodySim

INFO: Recompiling stale cache file /home/twan/code/RigidBodyDynamics/lib/v0.6/RigidBodySim.ji for module RigidBodySim.


`RigidBodySim` reexports all symbols from the following packages:
* [`RigidBodyDynamics`](https://github.com/tkoolen/RigidBodyDynamics.jl)
* [`RigidBodyTreeInspector`](https://github.com/rdeits/RigidBodyTreeInspector.jl)

In addition, it reexports select symbols from packages in the [`DifferentialEquations`](https://github.com/JuliaDiffEq/DifferentialEquations.jl) ecosystem. To access additional `DifferentialEquations` features, simply add one or more of the following:
```julia
using DiffEqBase # basics
using OrdinaryDiffEq # more ODE-related functionality
using DiffEqCallbacks # callback library
using DifferentialEquations # everything
```

# Model definition

We first need to create a `RigidBodyDynamics.Mechanism` instance, representing the rigid body system we want to simulate. One way of creating a `Mechanism` is by parsing a [URDF](http://wiki.ros.org/urdf) file:

In [2]:
urdf = Pkg.dir("RigidBodySim", "test", "urdf", "Acrobot.urdf")
mechanism = parse_urdf(Float64, urdf)
remove_fixed_tree_joints!(mechanism);

Next, we'll create a `MechanismState` object, and set the initial joint positions and velocities:

In [3]:
state = MechanismState(mechanism)
shoulder, elbow = joints(mechanism)
configuration(state, shoulder)[:] = 0.3
configuration(state, elbow)[:] = 0.4
velocity(state, shoulder)[:] = 1.
velocity(state, elbow)[:] = 2.;

# Creating an `ODEProblem`:

`RigidBodySim` uses RigidBodyDynamics.jl to evaluate the equations of motion, and [OrdinaryDiffEq.jl](https://github.com/JuliaDiffEq/OrdinaryDiffEq.jl) for numerical integration (see [DifferentialEquations.jl](https://github.com/JuliaDiffEq/DifferentialEquations.jl) for documentation).

`RigidBodySim` does not attempt to abstract away this dependence on the `DifferentialEquations` ecosystem, as doing so would necessarily expose only a subset of the functionality provided by `DifferentialEquations`, and require users familiar with the `DifferentialEquations` ecosystem to learn yet another API. Instead, `RigidBodySim` simply plugs into existing `DifferentialEquations` functionality, providing convenience methods and extensions.

One example of such a convience method is the following `DiffEqBase.ODEProblem` constructor, which sets up (but doesn't solve) the initial value problem to be solved through numerical integration of the equations of motion:

In [4]:
final_time = 10000.
problem = ODEProblem(state, (0., final_time))

DiffEqBase.ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 10000.0)
u0: [0.3, 0.4, 1.0, 2.0]

# Visualization

`RigidBodySim` uses `RigidBodyTreeInspector` for visualization.

We'll begin by starting a new `DrakeVisualizer` process (a separate visualizer window) if one isn't running already:

In [5]:
any_open_visualizer_windows() || (new_visualizer_window(); sleep(1));

For those familiar with `DrakeVisualizer`, note that `RigidBodySim.new_visualizer_window()` is different from `DrakeVisualizer.new_window()`: `RigidBodySim` provides a customized visualizer, with buttons that allow the user to interact with the simulation.

Next, we'll create a `DrakeVisualizer.Visualizer` for our mechanism, a handle which allows us to interact with the visualizer window. We'll then visualize the mechanism at the chosen initial state:

In [6]:
vis = Visualizer(mechanism, parse_urdf(urdf, mechanism))
settransform!(vis, state);

# Simulating while visualizing

We don't just want to visualize the initial state; we want to visualize the mechanism during the simulation as well. In addition, we want to be able to interact with the simulation process (for example, by terminating it using the 'stop' button in the visualizer).

This functionality is implemented using a set of `DifferentialEquations` integrator callbacks (i.e., a `DiffEqBase.CallbackSet`). `RigidBodySim` provides a convenience `CallbackSet` constructor for these visualizer hooks:

In [7]:
vis_callbacks = CallbackSet(vis, state);

We can now start simulating by simply calling the `DiffEqBase.solve` function, making sure to pass in our visualizer callback set as the `callback` keyword argument.

**Note**: just in time compilation may make it seem like nothing is happening for a few seconds.

In [8]:
sol = solve(problem, Tsit5(), abs_tol = 1e-10, dt = 0.05, callback = vis_callbacks);

**Note**: to terminate the simulation early, press the 'stop' button in the visualizer. If you'd like to simulate indefinitely, until the stop button is pressed, simply set `final_time` above to `Inf`.

**Note**: a headless simulation can of course be performed by simply omitting the `callback` keyword argument. This may be somewhat faster than visualizing while simulating.

# Visualization after the simulation

We can also play back the simulation results after the simulation is completed. RigidBodySim provides an overload for `RigidBodyTreeInspector.animate` that takes a `DiffEqBase.ODESolution`, returned by the `solve` function. Note that this `animate` function can be used even if the original simulation was performed headless (without visualization).

Here's an example 5-second headless simulation, followed by playback at half speed:

In [9]:
problem = ODEProblem(state, (0., 5.))
sol = solve(problem, Vern7(), abs_tol = 1e-10, dt = 0.05);

In [10]:
animate(vis, state, sol; realtime_rate = 0.5)

Like with simulation, the 'stop' button in the visualizer window can be used to terminate visualization early.

# Simulation with a controller in the loop

So far, we've only simulated a passive mechanism. We can also simulate with a controller in the loop.

## Basics

The basic approach to controller-in-the-loop simulation is to pass in an a function of the form
```julia
function (τ::AbstractVector, t, state::MechanismState)
    # set τ based on t and state...
end
```
as an extra argument to the `ODEProblem` constructor. For example:

In [11]:
function control!(τ, t, state)
    view(τ, velocity_range(state, shoulder))[:] = sin(t)
    view(τ, velocity_range(state, elbow)) .= -configuration(state, shoulder)
end

control! (generic function with 1 method)

In [12]:
controlproblem = ODEProblem(state, (0., 5.), control!)
sol = solve(controlproblem, Vern7(), abs_tol = 1e-10, dt = 0.05)
animate(vis, state, sol)

## Simulating a digital controller

In the previous example, the function `control!` is called whenever the integrator queries the dynamics. There are cases where this is not desired. For example, a digital controller typically runs at some fixed frequency, independent of the integrator time step. For such cases, `RigidBodySim` provides the `PeriodicController` type:

In [13]:
τ = similar(velocity(state))
Δt = 1e-3 # time between calls to the `control!` function
controller = PeriodicController(τ, Δt, control!);

which can be passed into the `ODEProblem` constructor in similar fashion:

In [14]:
periodic_control_problem = ODEProblem(state, (0., 5.), controller)

DiffEqBase.ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 5.0)
u0: [0.0121495, 0.471529, -1.43205, -0.50953]

Under the hood, this creates an `ODEProblem` that contains a [`DiffEqCallbacks.PeriodicCallback`](http://docs.juliadiffeq.org/stable/features/callback_library.html#PeriodicCallback-1), which ensures that the `control!` function is called at the desired rate.

In [15]:
sol = solve(periodic_control_problem, Tsit5(), abs_tol = 1e-10, dt = 0.05, callback = vis_callbacks);

and animate the results:

In [16]:
animate(vis, state, sol)